In [32]:
from IPython.display import Image
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()

import warnings
warnings.simplefilter("ignore")

## EmazeMeGroups

In [33]:
groups = pd.read_csv('data/EmazeMeGroups.csv')

In [34]:
groups.head()

,GroupID,GroupName,Admin,Privecy,Visibility,priority,varified,AssociatedGroupsJson,ModeratorsJson,DateCreated,DateUpdated,AboutGroup,EmazeSite,GroupCoverUrl,GroupPhotoUrl,SiteThumnail
0,1,BeitarJerusalem,d1f13501-60bf-4dd8-b6e8-c11d2f8eab8c,1,0,0,0,NaN,NaN,2020-04-05 10:27:38.0531290,2020-04-05 10:27:38.0531290,Group of Beitar players,https://emaze.me/diogoverdasca,//userscontent2.emaze.com/userinfo//cover/2dc9...,//userscontent2.emaze.com/userinfo//image/7c4d...,//storage.googleapis.com/userscontent2.emaze.c...
1,2,BeitarFans,d1f13501-60bf-4dd8-b6e8-c11d2f8eab8c,1,0,0,0,NaN,NaN,2020-04-05 10:27:38.0531290,2020-04-05 10:27:38.0531290,Group for fans of Beitar Jerusalem f.c.,NaN,//userscontent2.emaze.com/userinfo//cover/b397...,//userscontent2.emaze.com/userinfo//image/1e5a...,NaN
2,3,newgroupwithdate2,NaN,0,0,0,0,NaN,NaN,2020-04-05 12:00:36.2022096,2020-04-05 12:00:36.2022096,NaN,NaN,NaN,NaN,NaN
3,4,another-group,NaN,0,0,0,0,NaN,NaN,2020-04-20 08:57:01.8046276,2020-04-20 08:57:01.8046276,NaN,NaN,NaN,NaN,NaN
4,5,"""""",NaN,0,0,0,0,NaN,NaN,2020-04-20 10:44:39.2241056,2020-04-20 10:44:39.2241056,NaN,NaN,NaN,NaN,NaN


In [35]:
groups = groups.rename(columns = {'GroupID':'EmazeMeGroupID'})

## EmazeMeGroupFollowers

In [36]:
group_f = pd.read_csv('data/EmazeMeGroupFollowers.csv')

In [37]:
group_f.count()

FollowId          106
FollowerId        106
EmazeMeGroupId    106
FollowingSince    106
dtype: int64

In [38]:
group_f.head()

,FollowId,FollowerId,EmazeMeGroupId,FollowingSince
0,3,240238a0-ce99-4ea7-ac77-eff09fb5eff2,1,0001-01-03 00:00:00.000
1,4,240238a0-ce99-4ea7-ac77-eff09fb5eff2,2,0001-01-03 00:00:00.000
2,5,e9477dec-c960-4f11-a563-9781cf8b1c15,1,0001-01-03 00:00:00.000
3,6,e9477dec-c960-4f11-a563-9781cf8b1c15,2,0001-01-03 00:00:00.000
4,7,93aff917-ec28-4201-b1d2-788e275671b2,1,0001-01-03 00:00:00.000


In [39]:
#create a values column for the matrix, make everything 1 since already followed
group_f['following'] = 1

In [40]:
#dictionary for groupid to groupname
item_dict = dict(zip(groups['EmazeMeGroupID'], groups['GroupName']))

## Create sparse FollowerId and Quantity matrix

In [41]:
group_matrix_df = group_f.pivot_table(
    values='following', 
    index='FollowerId', 
    columns='EmazeMeGroupId'
)

In [42]:
group_matrix_df.head()

EmazeMeGroupId,1,2,13,14,15,26,30,31,32,37,38
FollowerId,,,,,,,,,,,
09d32136-ed59-49fd-b553-5c1448aa92fd,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1c1994c7-9bb0-477d-83dd-dc0d6f822c10,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
240238a0-ce99-4ea7-ac77-eff09fb5eff2,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2415e720-8d4c-44c7-9cbf-cb1e76dcf4e8,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
24c1d879-cb12-4582-87b1-982c3b9ef4e8,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


To evaluate the recommender system mathematically, we are taking out 10 random 1.0's in the table, replacing them with NaN, and will see the matrix created by SVD and how those points compare to 1.0 

In [43]:
#evaluation, taking out 10 1.0s
group_matrix_df.iloc[2,1] = np.nan
group_matrix_df.iloc[19,0] = np.nan
group_matrix_df.iloc[22,0] = np.nan
group_matrix_df.iloc[26,1] = np.nan
group_matrix_df.iloc[26,2] = np.nan
group_matrix_df.iloc[53,2] = np.nan
group_matrix_df.iloc[49,1] = np.nan
group_matrix_df.iloc[49,0] = np.nan
group_matrix_df.iloc[48,2] = np.nan
group_matrix_df.iloc[48,3] = np.nan

In [44]:
follower_id_list = np.array(group_matrix_df.index.tolist())
group_id_list = np.array(group_matrix_df.columns.tolist())

In [45]:
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds

In [46]:
#turn the dataframe into a sparse matrix 
group_mtx = group_matrix_df.fillna(0).values.copy()
group_sparse_mtx = coo_matrix(group_mtx)

In [47]:
group_sparse_mtx

<62x11 sparse matrix of type '<class 'numpy.float64'>'
	with 96 stored elements in COOrdinate format>

## SVD of matrix

In [48]:
U, s, V = svds(group_sparse_mtx, k=1)

## Multiply the decompostion back to get approx matrix

In [49]:
#make sigma into a diagonal matrix
s_diag = np.diag(s)

In [50]:
group_svd = U @ s_diag @ V

In [51]:
group_svd

array([[1.78842656e-01, 1.62119765e-01, 2.30980510e-02, 5.06910908e-02,
        1.30789195e-01, 9.13631544e-01, 9.30066576e-03, 9.30066576e-03,
        9.30066576e-03, 9.30066576e-03, 9.30066576e-03],
       [1.78842656e-01, 1.62119765e-01, 2.30980510e-02, 5.06910908e-02,
        1.30789195e-01, 9.13631544e-01, 9.30066576e-03, 9.30066576e-03,
        9.30066576e-03, 9.30066576e-03, 9.30066576e-03],
       [7.05329323e-02, 6.39376683e-02, 9.10953407e-03, 1.99918261e-02,
        5.15813488e-02, 3.60322941e-01, 3.66804678e-03, 3.66804678e-03,
        3.66804678e-03, 3.66804678e-03, 3.66804678e-03],
       [1.78842656e-01, 1.62119765e-01, 2.30980510e-02, 5.06910908e-02,
        1.30789195e-01, 9.13631544e-01, 9.30066576e-03, 9.30066576e-03,
        9.30066576e-03, 9.30066576e-03, 9.30066576e-03],
       [1.78842656e-01, 1.62119765e-01, 2.30980510e-02, 5.06910908e-02,
        1.30789195e-01, 9.13631544e-01, 9.30066576e-03, 9.30066576e-03,
        9.30066576e-03, 9.30066576e-03, 9.30066576e-

In [52]:
#divide each row in matrix by the max number in the row to normalize values, making 1 the highest
for i in range(len(group_svd)):
    for j in range(len(group_svd[i])):
        group_svd[i][j] = group_svd[i][j] / max(group_svd[i])

In [53]:
group_svd

array([[0.19574921, 0.17744546, 0.02528158, 0.05548308, 0.14315311,
        1.        , 0.00930067, 0.00930067, 0.00930067, 0.00930067,
        0.00930067],
       [0.19574921, 0.17744546, 0.02528158, 0.05548308, 0.14315311,
        1.        , 0.00930067, 0.00930067, 0.00930067, 0.00930067,
        0.00930067],
       [0.19574921, 0.17744546, 0.02528158, 0.05548308, 0.14315311,
        1.        , 0.00366805, 0.00366805, 0.00366805, 0.00366805,
        0.00366805],
       [0.19574921, 0.17744546, 0.02528158, 0.05548308, 0.14315311,
        1.        , 0.00930067, 0.00930067, 0.00930067, 0.00930067,
        0.00930067],
       [0.19574921, 0.17744546, 0.02528158, 0.05548308, 0.14315311,
        1.        , 0.00930067, 0.00930067, 0.00930067, 0.00930067,
        0.00930067],
       [0.19574921, 0.17744546, 0.02528158, 0.05548308, 0.14315311,
        1.        , 0.00347096, 0.00347096, 0.00347096, 0.00347096,
        0.00347096],
       [0.19574921, 0.17744546, 0.02528158, 0.05548308, 0.

In [54]:
#looking up the points that we took out earlier and seeing how they compare to 1.0, their actual value
prediction = list([group_svd[2][1],
    group_svd[19][0],
    group_svd[22][0],
    group_svd[26][1],
    group_svd[26][2],
    group_svd[53][2],
    group_svd[49][1],
    group_svd[49][0],
    group_svd[48][2],
    group_svd[48][3]])

In [55]:
actual = list([1,1,1,1,1,1,1,1,1,1])

In [56]:
from sklearn import metrics
rmse = (metrics.mean_squared_error(actual,prediction))
print(rmse)

0.7712586118281175


In each of the rows, the max number is 1, and then the next highest number is usually around 0.19, so a rmse of 0.77 is exceptional and shows the accuracy of the rec system.

# Rec Function

In [57]:
col = ["user_id","rec_group_id", "rec_group_name"]
rows = []
def recommend(follower_list, num_results=3):
    for follower_id in follower_list:
        
        # get the row index of follower
        follower_indx = np.where(follower_id_list == follower_id)[0][0]
        
        # sort all vales for follower row in descending order
        indx_sort = group_svd[follower_indx, :].argsort()[::-1]
        
        # create mask and get the index of only groups never followed
        never_follow = group_mtx[follower_indx, :][indx_sort] == 0
    
        rec_indx = indx_sort[never_follow]
        rec_ids = group_id_list[rec_indx]
        rec_name = [item_dict[x] for x in rec_ids]
        rows.append([follower_id,rec_ids[:num_results],rec_name[:num_results]])

In [58]:
#recommend function takes in a list of users, and outputs a dataframe with the users and their recommendations
#recommend(['09d32136-ed59-49fd-b553-5c1448aa92fd','b6d10dc9-189f-496d-9956-af760c43278b','8f138dce-bc62-4c97-ad8c-d008662810f6','db8d639f-7c6a-432e-8e43-07ea98b41607','240238a0-ce99-4ea7-ac77-eff09fb5eff2'])
recommend(follower_id_list)

In [59]:
reco_df = pd.DataFrame(rows, columns=col)

In [63]:
reco_df.head(10)

,user_id,rec_group_id,rec_group_name
0,09d32136-ed59-49fd-b553-5c1448aa92fd,"[1, 2, 15]","[BeitarJerusalem, BeitarFans, Israeli Singers]"
1,1c1994c7-9bb0-477d-83dd-dc0d6f822c10,"[1, 2, 15]","[BeitarJerusalem, BeitarFans, Israeli Singers]"
2,240238a0-ce99-4ea7-ac77-eff09fb5eff2,"[26, 2, 13]","[Soccer Players, BeitarFans, Emaze]"
3,2415e720-8d4c-44c7-9cbf-cb1e76dcf4e8,"[1, 2, 15]","[BeitarJerusalem, BeitarFans, Israeli Singers]"
4,24c1d879-cb12-4582-87b1-982c3b9ef4e8,"[1, 2, 15]","[BeitarJerusalem, BeitarFans, Israeli Singers]"
5,2656ebcc-1919-4bbb-b34c-ffbdbfc2082b,"[26, 15, 14]","[Soccer Players, Israeli Singers, Design and M..."
6,278231bc-75dd-4fcf-a610-36ddc1df8455,"[1, 2, 15]","[BeitarJerusalem, BeitarFans, Israeli Singers]"
7,3199cab4-ee6d-4a7e-b629-7c4adf5f5fb4,"[1, 2, 15]","[BeitarJerusalem, BeitarFans, Israeli Singers]"
8,371088ec-ccd9-4c1e-acb3-6c3b43c84627,"[26, 15, 14]","[Soccer Players, Israeli Singers, Design and M..."
9,38dee454-fd00-4a2f-937d-e10c74b27b42,"[1, 26, 2]","[BeitarJerusalem, Soccer Players, BeitarFans]"


#### Uncomment line below to export file into csv

In [61]:
#reco_df.to_csv('recommend_groups.csv')